# Sifferigenkänning

In [ ]:
# Jag antar att tensorflow redan är installerat
#!pip install tensorflow

## MNIST dataset

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
train_labels

In [ ]:
test_images.shape

In [ ]:
len(test_labels)

In [ ]:
test_labels

In [ ]:
# plotta de 3 första siffrorna i träningsdata
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
plt.subplot(1, 3, 1)
plt.imshow(train_images[0], cmap='gray')
plt.subplot(1, 3, 2)
plt.imshow(train_images[1], cmap='gray')
plt.subplot(1, 3, 3)
plt.imshow(train_images[2], cmap='gray')
plt.show()

## Enkel "fully connected" modell 

In [ ]:
# definiera modellen
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [ ]:
# kompilera modellen
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# preparera träningsdata och testdata
# gör om (28,28) matrisen till en vektor (28*28)
train_images = train_images.reshape((60000, 28 * 28))
# pixelvärdena blir mellan 0.0 och 1.0
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
print(train_images.shape)
print(test_images.shape)

In [ ]:
# träna modellen med 128 bilder i taget. Kör igenom alla data 5 gånger
model.fit(train_images, train_labels, epochs=5, batch_size=128)

In [ ]:
model.summary()

In [ ]:
# beräkna noggrannheten för testdata
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
# om vi ger bara en bild till vår modell måste vi lägga till en batchdimension
# predict ger en array med sannolikheter för siffrorna 0-9
# argmax() ger index för det största värdet
import numpy as np
print(test_images[0].shape)
image = test_images[0][np.newaxis]
print(image.shape)
predictions = model.predict(image)
print(predictions)
print("prediktion:" + str(predictions[0].argmax()) + " sannolikhet:" + str(predictions[0].max()))
print("label:" + str(test_labels[0]))

In [ ]:
# om vi ger flera bilder till modellen finns redan en batchdimension
predictions = model.predict(test_images[0:10])
for i in range(len(predictions)):
    print("prediktion:" + str(predictions[i].argmax()) + " label:" + str(test_labels[i]))

## Enkelt CNN

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
# ladda och preparera data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
print(train_images.shape)
print(test_images.shape)

In [ ]:
# kompilera modellen
model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [ ]:
# träna modellen med 64 bilder i taget. Kör igenom alla data 5 gånger
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
model.save("mnist_classify")

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

# Potta Feature Maps

In [ ]:
# definiera activation_model
# plockar ut aktiveringarna från vår CNN-modell
from tensorflow.keras import layers
layer_outputs = []
layer_names = []
# skapa lista med aktiveringar
for layer in model.layers:
    if isinstance(layer, (layers.Conv2D, layers.MaxPooling2D)):
        layer_outputs.append(layer.output)
        layer_names.append(layer.name)
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
plt.imshow(train_images[1], cmap='gray')

In [ ]:
# skapa aktiveringar/feature maps för vår siffra
activations = activation_model.predict(train_images[1][np.newaxis])

In [ ]:
for i in range(len(activations)):
    print(str(i) + ': ', activations[i].shape)

In [ ]:
for i in range(len(activations)):
    sh = activations[i].shape
    print('-----------------------------------------------')
    print(layer_names[i])
    print(str(sh[3]) + ' feature maps ' + str(sh[1]) + 'x' + str(sh[2]))
    for j in range(sh[3]):
        print('map' + str(j) + ' av ' + str(sh[3]-1))
        plt.axis('off')
        plt.imshow(activations[i][0,:,:,j], cmap='gray')
        plt.show()

In [ ]:
# plotta första feature map för lagren
plt.figure(figsize=(12, 8))
for i in range(len(activations)):
    sh = activations[i].shape
    plt.subplot(2, 3, i + 1)
    plt.axis('off')
    plt.title(layer_names[i] + ' ' + str(sh[1]) + 'x' + str(sh[2]))
    plt.imshow(activations[i][0,:,:,0], cmap='gray')
plt.show()

# Felaktig eller tveksam label?
### Om label är fel SÄTT LABEL = PREDIKTION
### [Eventuellt] Om det är omöjligt att avgöra vilken siffra SÄTT LABEL = PREDIKTION

In [ ]:
def corr_labels(data, labels):
    ncorr = 0
    nfel = 0
    for i in range(len(data)):
        image = data[i][np.newaxis]
        predikt = model.predict(image).argmax()
        if (predikt != labels[i]):
            nfel = nfel + 1
            print('-------------------------------------------------------')
            plt.imshow(data[i], cmap='gray')
            plt.show()
            print("Bild nr:" + str(i))
            print("Labeled:" + str(labels[i]))
            print("Prediktion:" + str(predikt))
            digit = input("Enter correct digit or return if label is correct:")
            if (len(digit) > 0):
                digit = int(digit)
                if (digit >=0 and digit <= 9):
                    labels[i] = digit
                    ncorr = ncorr + 1
    print(str(nfel) + "felaktiga prediktioner")
    print(str(ncorr) + "labels ändrade")

In [ ]:
# Kolla labels för träningsdata
# Kolla en bit för att se helt felaktiga labels. Jag hittade några...
# Klicka till vänster i cellen för att slippa scroll bars
corr_labels(train_images, train_labels)

In [ ]:
# Kolla labels för testdata
corr_labels(test_images, test_labels)